# Libraries

In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import xgboost

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

import warnings
warnings.simplefilter('ignore')

In [ ]:
input_path = "../input/tabular-playground-series-jun-2022"

df = pd.read_csv("/".join([input_path, "data.csv"]), index_col='row_id')
submission = pd.read_csv("/".join([input_path, "sample_submission.csv"]), index_col='row-col')

# IterativeImputer

In [ ]:
imputed_list = []
n_seeds = 3

for seed in tqdm(range(n_seeds)):
    imputer = IterativeImputer(
        estimator=xgboost.XGBRegressor(
            n_estimators=2000,
            random_state=seed,
            tree_method='gpu_hist',
            predictor='gpu_predictor'
        ),
        missing_values=np.nan,
        max_iter=1,
        initial_strategy='mean',
        imputation_order='ascending',
        verbose=2,
        random_state=seed
    )

    imputed_list.append(pd.DataFrame(imputer.fit_transform(df),
                                     columns = df.columns)
                       )

# Submission

In [ ]:
for idx in tqdm(submission.index):
    row = int(idx.split('-')[0])
    col = idx.split('-')[1]
    tmp = 0
    for i in range(n_seeds):
        tmp += imputed_list[i].loc[row, col] / n_seeds
    submission.loc[idx, 'value'] = tmp
        
submission.to_csv("submission.csv")
submission